In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
import numpy as np
import random
import os

In [0]:
os.getcwd()

'/content/drive/dinosaur'

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
os.chdir("/content/drive/dinosaur")

In [0]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


In [0]:
def smooth(loss, cur_loss):
    return loss * 0.999 + cur_loss * 0.001


In [0]:
def print_sample(sample_ix, ix_to_char):
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    txt = txt[0].upper() + txt[1:]  # capitalize first character 
    print ('%s' % (txt, ), end='')

In [0]:
def get_initial_loss(vocab_size, seq_length):
    return -np.log(1.0/vocab_size)*seq_length

In [0]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


In [0]:
def initialize_parameters(n_a, n_x, n_y):
    """
    Initialize parameters with small random values
    
    Returns:
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        b --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    """
    np.random.seed(1)
    Wax = np.random.randn(n_a, n_x)*0.01 # input to hidden
    Waa = np.random.randn(n_a, n_a)*0.01 # hidden to hidden
    Wya = np.random.randn(n_y, n_a)*0.01 # hidden to output
    b = np.zeros((n_a, 1)) # hidden bias
    by = np.zeros((n_y, 1)) # output bias
    
    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b,"by": by}
    
    return parameters


In [0]:
def rnn_step_forward(parameters, a_prev, x):
    
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    a_next = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b) # hidden state
    p_t = softmax(np.dot(Wya, a_next) + by) # unnormalized log probabilities for next chars # probabilities for next chars 
    
    return a_next, p_t


In [0]:
def rnn_step_backward(dy, gradients, parameters, x, a, a_prev):
    
    gradients['dWya'] += np.dot(dy, a.T)
    gradients['dby'] += dy
    da = np.dot(parameters['Wya'].T, dy) + gradients['da_next'] # backprop into h
    daraw = (1 - a * a) * da # backprop through tanh nonlinearity
    gradients['db'] += daraw
    gradients['dWax'] += np.dot(daraw, x.T)
    gradients['dWaa'] += np.dot(daraw, a_prev.T)
    gradients['da_next'] = np.dot(parameters['Waa'].T, daraw)
    return gradients


In [0]:
def update_parameters(parameters, gradients, lr):

    parameters['Wax'] += -lr * gradients['dWax']
    parameters['Waa'] += -lr * gradients['dWaa']
    parameters['Wya'] += -lr * gradients['dWya']
    parameters['b']  += -lr * gradients['db']
    parameters['by']  += -lr * gradients['dby']
    return parameters


In [0]:
def rnn_forward(X, Y, a0, parameters, vocab_size = 27):
    
    # Initialize x, a and y_hat as empty dictionaries
    x, a, y_hat = {}, {}, {}
    
    a[-1] = np.copy(a0)
    
    # initialize your loss to 0
    loss = 0
    
    for t in range(len(X)):
        
        # Set x[t] to be the one-hot vector representation of the t'th character in X.
        # if X[t] == None, we just have x[t]=0. This is used to set the input for the first timestep to the zero vector. 
        x[t] = np.zeros((vocab_size,1)) 
        if (X[t] != None):
            x[t][X[t]] = 1
        
        # Run one step forward of the RNN
        a[t], y_hat[t] = rnn_step_forward(parameters, a[t-1], x[t])
        
        # Update the loss by substracting the cross-entropy term of this time-step from it.
        loss -= np.log(y_hat[t][Y[t],0])
        
    cache = (y_hat, a, x)
        
    return loss, cache


In [0]:
def rnn_backward(X, Y, parameters, cache):
    # Initialize gradients as an empty dictionary
    gradients = {}
    
    # Retrieve from cache and parameters
    (y_hat, a, x) = cache
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    
    # each one should be initialized to zeros of the same dimension as its corresponding parameter
    gradients['dWax'], gradients['dWaa'], gradients['dWya'] = np.zeros_like(Wax), np.zeros_like(Waa), np.zeros_like(Wya)
    gradients['db'], gradients['dby'] = np.zeros_like(b), np.zeros_like(by)
    gradients['da_next'] = np.zeros_like(a[0])
    
    # Backpropagate through time
    for t in reversed(range(len(X))):
        dy = np.copy(y_hat[t])
        dy[Y[t]] -= 1
        gradients = rnn_step_backward(dy, gradients, parameters, x[t], a[t], a[t-1])
    
    return gradients, a



In [0]:
data = open('dinos.txt', 'r').read()
data= data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))
#print(data)

There are 19909 total characters and 27 unique characters in your data.


In [0]:
!ls

dinos.txt  drive  Untitled0.ipynb


In [0]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(char_to_ix)
print(ix_to_char)

{'\n': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


In [0]:
#clip

def clip(gradients, maxValue):
    '''
    Clips the gradients' values between minimum and maximum.
    
    Arguments:
    gradients -- a dictionary containing the gradients "dWaa", "dWax", "dWya", "db", "dby"
    maxValue -- everything above this number is set to this number, and everything less than -maxValue is set to -maxValue
    
    Returns: 
    gradients -- a dictionary with the clipped gradients.
    '''
    
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    # clip to mitigate exploding gradients, loop over [dWax, dWaa, dWya, db, dby]. (≈2 lines)
    for gradient in [dWax, dWaa, dWya, db, dby]:
        np.clip(gradient,-maxValue,maxValue,gradient)
    
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients

In [0]:
# sample

def sample(parameters, char_to_ix, seed):
    """
    Sample a sequence of characters according to a sequence of probability distributions output of the RNN

    Arguments:
    parameters -- python dictionary containing the parameters Waa, Wax, Wya, by, and b. 
    char_to_ix -- python dictionary mapping each character to an index.
    seed -- used for grading purposes. Do not worry about it.

    Returns:
    indices -- a list of length n containing the indices of the sampled characters.
    """
    
    # Retrieve parameters and relevant shapes from "parameters" dictionary
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    vocab_size = by.shape[0]
    n_a = Waa.shape[1]
    
    # Step 1: Create the one-hot vector x for the first character (initializing the sequence generation). (≈1 line)
    x = np.zeros((vocab_size, 1))
    # Step 1': Initialize a_prev as zeros (≈1 line)
    a_prev = np.zeros((n_a, 1))
    
    # Create an empty list of indices, this is the list which will contain the list of indices of the characters to generate (≈1 line)
    indices = []
    
    # Idx is a flag to detect a newline character, we initialize it to -1
    idx = -1 
    
    # Loop over time-steps t. At each time-step, sample a character from a probability distribution and append 
    # its index to "indices". We'll stop if we reach 50 characters (which should be very unlikely with a well 
    # trained model), which helps debugging and prevents entering an infinite loop. 
    counter = 0
    newline_character = char_to_ix['\n']
    
    while (idx != newline_character and counter != 50):
        
        # Step 2: Forward propagate x using the equations (1), (2) and (3)
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b)
        z = np.dot(Wya, a) + by
        y = softmax(z)
        
        # for grading purposes
        np.random.seed(counter + seed) 
        
        # Step 3: Sample the index of a character within the vocabulary from the probability distribution y
        idx = np.random.choice(list(range(vocab_size)), p=y.ravel())

        # Append the index to "indices"
        indices.append(idx)
        
        # Step 4: Overwrite the input character as the one corresponding to the sampled index.
        x = np.zeros((vocab_size, 1))
        x[idx] = 1
        
        # Update "a_prev" to be "a"
        a_prev = a
        
        # for grading purposes
        seed += 1
        counter +=1
        
    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    return indices

In [0]:
#  optimize

def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    """
    Execute one step of the optimization to train the model.
    
    Arguments:
    X -- list of integers, where each integer is a number that maps to a character in the vocabulary.
    Y -- list of integers, exactly the same as X but shifted one index to the left.
    a_prev -- previous hidden state.
    parameters -- python dictionary containing:
                        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
                        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
                        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
                        b --  Bias, numpy array of shape (n_a, 1)
                        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
    learning_rate -- learning rate for the model.
    
    Returns:
    loss -- value of the loss function (cross-entropy)
    gradients -- python dictionary containing:
                        dWax -- Gradients of input-to-hidden weights, of shape (n_a, n_x)
                        dWaa -- Gradients of hidden-to-hidden weights, of shape (n_a, n_a)
                        dWya -- Gradients of hidden-to-output weights, of shape (n_y, n_a)
                        db -- Gradients of bias vector, of shape (n_a, 1)
                        dby -- Gradients of output bias vector, of shape (n_y, 1)
    a[len(X)-1] -- the last hidden state, of shape (n_a, 1)
    """
    
    # Forward propagate through time (≈1 line)
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    # Backpropagate through time (≈1 line)
    gradients, a = rnn_backward(X, Y, parameters, cache)
    
    # Clip your gradients between -5 (min) and 5 (max) (≈1 line)
    gradients = clip(gradients, 5)
    
    # Update parameters (≈1 line)
    parameters = update_parameters(parameters, gradients, learning_rate)
    
    
    return loss, gradients, a[len(X)-1]

In [0]:
# GRADED FUNCTION: model

def model(data, ix_to_char, char_to_ix, num_iterations = 1000000, n_a = 50, dino_names = 7, vocab_size = 27):
    """
    Trains the model and generates dinosaur names. 
    
    Arguments:
    data -- text corpus
    ix_to_char -- dictionary that maps the index to a character
    char_to_ix -- dictionary that maps a character to an index
    num_iterations -- number of iterations to train the model for
    n_a -- number of units of the RNN cell
    dino_names -- number of dinosaur names you want to sample at each iteration. 
    vocab_size -- number of unique characters found in the text, size of the vocabulary
    
    Returns:
    parameters -- learned parameters
    """
    
    # Retrieve n_x and n_y from vocab_size
    n_x, n_y = vocab_size, vocab_size
    
    # Initialize parameters
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    # Initialize loss (this is required because we want to smooth our loss, don't worry about it)
    loss = get_initial_loss(vocab_size, dino_names)
    
    # Build list of all dinosaur names (training examples).
    with open("dinos.txt") as f:
        examples = f.readlines()
    examples = [x.lower().strip() for x in examples]
    
    # Shuffle list of all dinosaur names
    np.random.seed(0)
    np.random.shuffle(examples)
    
    # Initialize the hidden state of your LSTM
    a_prev = np.zeros((n_a, 1))
    
    # Optimization loop
    for j in range(num_iterations):
        
        
        # Use the hint above to define one training example (X,Y) (≈ 2 lines)
        index = j % len(examples)
        X = [None] + [char_to_ix[ch] for ch in examples[index]] 
        Y = X[1:] + [char_to_ix["\n"]]
        
        # Perform one optimization step: Forward-prop -> Backward-prop -> Clip -> Update parameters
        # Choose a learning rate of 0.01
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters)
        
        
        # Use a latency trick to keep the loss smooth. It happens here to accelerate the training.
        loss = smooth(loss, curr_loss)

        # Every 2000 Iteration, generate "n" characters thanks to sample() to check if the model is learning properly
        if j % 2000 == 0:
            
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            
            # The number of dinosaur names to print
            seed = 0
            for name in range(dino_names):
                
                # Sample indices and print them
                sampled_indices = sample(parameters, char_to_ix, seed)
                print_sample(sampled_indices, ix_to_char)
                
                seed += 1  # To get the same result for grading purposed, increment the seed by one. 
      
            print('\n')
        
    return parameters

In [0]:
parameters = model(data, ix_to_char, char_to_ix)

Iteration: 0, Loss: 23.087336

Nkzxwtdmfqoeyhsqwasjkjvu
Kneb
Kzxwtdmfqoeyhsqwasjkjvu
Neb
Zxwtdmfqoeyhsqwasjkjvu
Eb
Xwtdmfqoeyhsqwasjkjvu


Iteration: 2000, Loss: 27.884160

Liusskeomnolxeros
Hmdaairus
Hytroligoraurus
Lecalosapaus
Xusicikoraurus
Abalpsamantisaurus
Tpraneronxeros


Iteration: 4000, Loss: 25.901815

Mivrosaurus
Inee
Ivtroplisaurus
Mbaaisaurus
Wusichisaurus
Cabaselachus
Toraperlethosdarenitochusthiamamumamaon


Iteration: 6000, Loss: 24.608779

Onwusceomosaurus
Lieeaerosaurus
Lxussaurus
Oma
Xusteonosaurus
Eeahosaurus
Toreonosaurus


Iteration: 8000, Loss: 24.070350

Onxusichepriuon
Kilabersaurus
Lutrodon
Omaaerosaurus
Xutrcheps
Edaksoje
Trodiktonus


Iteration: 10000, Loss: 23.844446

Onyusaurus
Klecalosaurus
Lustodon
Ola
Xusodonia
Eeaeosaurus
Troceosaurus


Iteration: 12000, Loss: 23.291971

Onyxosaurus
Kica
Lustrepiosaurus
Olaagrraiansaurus
Yuspangosaurus
Eealosaurus
Trognesaurus


Iteration: 14000, Loss: 23.382338

Meutromodromurus
Inda
Iutroinatorsaurus
Maca
Yusteratop

Iteration: 76000, Loss: 21.957251

Plusoranosaurus
Lola
Mutosaurus
Peecestedanossdiaus
Ystreoiophus
Feaisondelusaurus
Wus


Iteration: 78000, Loss: 21.713355

Niwusidon
Jice
Lutroeoimisscriparaptor
Ola
Ystraphosaurus
Ga
Ustanhosaurus


Iteration: 80000, Loss: 21.822720

Pattrodon
Komaacouicansaurus
Lustraposaurus
Pacaeosaurus
Yrosaurus
Edalosaurus
Ustaodon


Iteration: 82000, Loss: 21.898584

Oryxareollomus
Jicabcosaurus
Lutosaurus
Olaaerosasrus
Ystonedriandrosaurus
Ed
Wrokibosaurus


Iteration: 84000, Loss: 21.563660

Nivosaurus
Jieg
Lustriphosaurus
Necairopeetthylentimaueseupersijingospondylong
Ystriphosaurus
Gdalosaurus
Wrocerator


Iteration: 86000, Loss: 21.893345

Meuscolchus
Jiabalosaurus
Kustraphia
Mabalosaurus
Yrosaurus
Edalosaurus
Wriandromumanucholes


Iteration: 88000, Loss: 21.659152

Pitrosaurus
Momaacosaurus
Mtptodon
Pacaeosaurus
Ystriphosaurus
Ga
Wronogon


Iteration: 90000, Loss: 21.746246

Nivsitathosaurus
Jiceakosaurus
Lutosaurus
Ndabeson
Ystyeresodratos
Edaisihachus

Iteration: 154000, Loss: 22.038279

Nlussrerchodun
Lola
Lusor
Ngbagosaurus
Yssurnatopulterax
Eg
Vosaurus


Iteration: 156000, Loss: 21.587421

Nousosaurus
Mlegbaschebisaurus
Mtrsaurus
Nmbalosaurus
Ytromidosaurus
Ibamus
Utong


Iteration: 158000, Loss: 21.691672

Nivosaurus
Imacakpilaceratopsilodycepsaurus
Ktosaurus
Naa
Ytisaurus
Eeaeptodenutodinuracuratodantidin
Trodon


Iteration: 160000, Loss: 21.556288

Patosaurus
Long
Mutosaurus
Pacahosaurus
Ytosaurus
Ehaeptomaes
Utonia


Iteration: 162000, Loss: 21.926973

Rovosaurus
Nodealosaurus
Nuusaprus
Rlcamis
Ytroides
Ha
Troides


Iteration: 164000, Loss: 21.769531

Onyushengsodratrus
Midea
Mutroenetopterus
Oddadrondchus
Ytroerasaurus
Edamosaurus
Trodon


Iteration: 166000, Loss: 21.985106

Oluscolomephus
Mia
Mrospheithatitan
Odacerosaurus
Ystleimia
Fabaosaurus
Trolomerhylopn


Iteration: 168000, Loss: 21.695489

Rous
Nodabasaurus
Otusaurus
Rng
Wrus
Geabus
Truroosaurus


Iteration: 170000, Loss: 21.571231

Mewtosaurus
Gledcasaurus
Hyuusaurus

Iteration: 234000, Loss: 26.277462

Lispsaurus
Inca
Itus
Maea
Wshochanlavesaurus
Eia
Tosaurus


Iteration: 236000, Loss: 26.513380

Neytosaurus
Kelbaisopacesaurus
Lustohneoraveros
Nedahstiddosaurus
Wtromongolyeset
Ecaess
Topenatons


Iteration: 238000, Loss: 26.539736

Imusisaurus
Inedaiton
Itrtolichilurostasaurus
Ilbaiton
Vrosaurus
Hc
Rosaurus


Iteration: 240000, Loss: 26.584583

Mivtsohikor
Ingcagssacantisaurus
Iyushengoomuositarilhss
Medaisthechurs
Wytolomor
Gbahrs
Strgolosaurus


Iteration: 242000, Loss: 26.214156

Onytrs
Nleiacosaurus
Owyus
Oia
Vtosaurus
Icamosaurus
Rrodon


Iteration: 244000, Loss: 26.051135

Levtosaurus
Inacagps
Iussidonnlethsps
Lacafosaurus
Wusolomor
Ecagosaurus
Troeonosaurus


Iteration: 246000, Loss: 25.484014

Inytosaurus
Inacaisaurus
Itsisaurus
Ila
Wutohianmausaus
Ebaisaurus
Rosaurus


Iteration: 248000, Loss: 25.185995

Inyspras
Inga
Iuspolonosaurus
Ilaadpribantosaurus
Vrosaurus
Haagspidantosaurus
Ous


Iteration: 250000, Loss: 24.724515

Invutraqfor
Inga

Iteration: 312000, Loss: 22.684467

Ilutosaurus
Inabairus
Itospangia
Ibaafosaurus
Rispendia
Haaeosaurus
Itoheisaurus


Iteration: 314000, Loss: 22.931911

Iptrosaurus
Hhadahrokachynthenus
Huston
Ilaadosaurus
Strodon
Caadosaurus
Otoloksanus


Iteration: 316000, Loss: 22.598448

Inus
Helcakosaurus
Hutstaohinoteratar
Ieeagror
Vptomimis
Baagosaurus
Prodon


Iteration: 318000, Loss: 22.796665

Ilutosaurus
Ileeagosaurus
Itosaurus
Igaagosaurus
Roshendosaurus
Haagosaurus
Osodon


Iteration: 320000, Loss: 22.582019

Inusosaurus
Inee
Itrosaurus
Igaaerukechylterotindyigtodetroenia
Rrorapropeunorochelior
Haaerukechylterotindyigtodetroenia
Otrangopeveotochelior


Iteration: 322000, Loss: 22.749271

Hextroenasceuhusaurus
Enacantor
Ewstong
Hadalosaurus
Vtrodon
Abalosaurus
Ptoloplonyshitasaurus


Iteration: 324000, Loss: 22.645063

Iluses
Inecahosaurus
Itrosaurus
Igaadosaurus
Ririangorgylosaurus
Haadosaurus
Otoendosaurus


Iteration: 326000, Loss: 22.716751

Inushnaneosaurus
Iracaeosaurus
Itrosaurus
I

...

Iteration: 630000, Loss: 21.855676

Inwtosaurus
Ipag
Itotophasaurus
Ilaacosaurus
Utosaurus
Acahtorachyptor
Itolia


Iteration: 632000, Loss: 22.140265

Inus
Iphaacosaurus
Itosaurus
Ileabosaurus
Qtosaurus
Acalosaurus
Itonikosaurus


Iteration: 634000, Loss: 21.823517

Itrosaurus
Iticagtidachyrophothiaxaurus
Itrosaurus
Ileaisthachuisaurus
Ptrolrarlitasaurus
Aladrope
Itrangosaurus


Iteration: 636000, Loss: 22.018662

Imusisaurus
Imia
Itrosaurus
Ila
Itrohamosaurus
Eiacosaurus
Itodon


Iteration: 638000, Loss: 22.033160

Imusaurus
Ingaa
Itrosaurus
Iicagtria
Utromimus
Acaissaurus
Itodon


Iteration: 640000, Loss: 21.716212

Ilusaurus
Atelalosaurus
Evosaurus
Ida
Utosaurus
Acaeosaurus
Ipodon


Iteration: 642000, Loss: 21.845097

Ilus
Imee
Itrosaurus
Ihaadrope
Utosaurus
Iaadrope
Itodon


Iteration: 644000, Loss: 22.023176

Imwusaurus
Higa
Iwutoiceratops
Iga
Utysaurus
Acakropa
Itrangikauisaurus


Iteration: 646000, Loss: 22.091268

Isysaurus
Itikansopagnsasaurus
Itrus
Ileacosaurus
Utrong
Acalsi

Iteration: 710000, Loss: 21.652757

Intosaurus
Inga
Itrosaurus
Ihaagosaurus
Prosaurus
Acaisauroptitalitirax
Itohelosaurus


Iteration: 712000, Loss: 21.627726

Amusis
Angaagosaurus
Austolecnnathus
Ala
Rtroikophotappocophosaurus
Acalosaurus
Ispong


Iteration: 714000, Loss: 21.345737

Inusaurus
Ingaa
Itrosaurus
Ile
Rstrionosaurus
Acaisrus
Istes


Iteration: 716000, Loss: 21.569527

Inus
Ingcanpton
Itosaurus
Igabasaurus
Utwodon
Acalosaurus
Ipsandorasaurus


Iteration: 718000, Loss: 21.678498

Enyusaurus
Aplaagusaurus
Atrosaurus
Ekaaisteia
Ptisaurus
Acalroraisuchus
Istangosaurus


Iteration: 720000, Loss: 21.411386

Atuskong
Atia
Atrosaurus
Anaaeosaurus
Itrophisaurus
Adaishafansaurus
Itoloplodrasaurus


Iteration: 722000, Loss: 21.509686

Ilus
Aphaadosaurus
Atrosaurus
Idaadosaurus
Ptosaurus
Acaertadersaurus
Irodon


Iteration: 724000, Loss: 21.872956

Ilus
Imecagtosaurus
Itrus
Idabdroradosaurus
Utrohelta
Cacartegansaurus
Iprephingvenitasaurus


Iteration: 726000, Loss: 21.602963

Aovtrodo

Iteration: 792000, Loss: 21.688841

Anyseuhchinevenatarangus
Apcaalosaurus
Austras
Ama
Utosaurus
Acagton
Ipter


Iteration: 794000, Loss: 21.664590

Ilusaurus
Imeg
Iutrodon
Igaakpticantholenyradzalopcestinmallisaurus
Utorapsaurus
Acalosaurus
Ipteratops


Iteration: 796000, Loss: 21.558179

Imutosaurus
Incaaissa
Iwosaurus
Ilaacitag
Tpronia
Ecaisaurus
Isteratops


Iteration: 798000, Loss: 21.590901

Amushralinopteryx
Anda
Austognathothtosaurus
Akaadosaurus
Utosaurus
Acaisrus
Irochmisaurus


Iteration: 800000, Loss: 21.392430

Anzosaurus
Anadannura
Atrosaurus
Amaaishanarornathus
Utosaurus
Ahagpicachmathamus
Iprasaurus


Iteration: 802000, Loss: 21.702664

Imusaurus
Inacachus
Ivrosaurus
Ihaaaton
Prosaurus
Acaisaurops
Ipmarapteus


Iteration: 804000, Loss: 21.857947

Anysisaurus
Ancaaeroma
Atrosaurus
Ama
Ptosaurus
Ahaeosaurus
Ips


Iteration: 806000, Loss: 21.428906

Inus
Apia
Atrosaurus
Ilebestelasopongothodtaosteitosaurus
Ptosaurus
Aeachsaurus
Itolelosaurus


Iteration: 808000, Loss: 21.5

Iteration: 874000, Loss: 21.229904

Ilusaurus
Angaagosaurus
Butosaurus
Ilaacosaurus
Utosaurus
Acaeosaurus
Iosaurus


Iteration: 876000, Loss: 21.252862

Imustrlomimosaurus
Angaadosaurus
Atrosaurus
Idia
Utysaurus
Ajalosaurus
Kutgraptosaurus


Iteration: 878000, Loss: 21.284728

Imusaurus
Angaa
Assosaurus
Ilcamosaurus
Utosaurus
Amanosaurus
Ioraptor


Iteration: 880000, Loss: 21.277273

Ilus
Ilia
Itporaptor
Ihaa
Utrocephelurus
Acalopicalunolopsalex
Poraptor


Iteration: 882000, Loss: 21.172204

Imushrasaurus
Inda
Iutrodon
Ila
Utrodon
Acaiscedarsaurus
Osteodon


Iteration: 884000, Loss: 21.393974

Anvusaurus
Anacacosaurus
Aushognathaudontasaurus
Ama
Utosaurus
Acacosaurus
Ipoceratops


Iteration: 886000, Loss: 21.062133

Imus
Indaaeosaurus
Ivosaurus
Ileakromaceratops
Utor
Acalor
Isperepimus


Iteration: 888000, Loss: 21.383338

Imuseurus
Inedansicelta
Iwotisaurus
Ikaadops
Utrokeiosaurus
Abaisaura
Irokeiosaurus


Iteration: 890000, Loss: 21.611080

Imusetaphosaurus
Enda
Hysaurus
Ileadops
Uto

Iteration: 956000, Loss: 21.269374

Inxusaurus
Anda
Atusaurus
Ilaadrun
Utrodon
Akagus
Utioperatops


Iteration: 958000, Loss: 20.974253

Anysaurus
Aphaadophachus
Atrosaurus
Acaadophachus
Utyran
Acaintheidonpides
Itocephalus


Iteration: 960000, Loss: 21.284061

Ilusaurus
Imia
Iutromemitathus
Ile
Utssareosaurus
Acalosaurus
Psphiceratops


Iteration: 962000, Loss: 21.160981

Ilusaurus
Ingaberrkaceratops
Itrysaurus
Igaceratops
Utyran
Acalosaurus
Utphiceratops


Iteration: 964000, Loss: 21.264163

Imusaurus
Gnadaisauropterosaurus
Hyskondosaurus
Igabasauropterosaurus
Uvronosaurus
Acalosaurus
Uvomimus


Iteration: 966000, Loss: 21.218439

Imusestenosaurus
Inecacsorhensaurus
Itoseeonoshynosectelisiantitepanentelecopalicosaur
Ilecerosaurus
Utosaurus
Acalosaurus
Ptolopnolsaurus


Iteration: 968000, Loss: 21.227668

Imus
Ingcensuchmsilolesuchsuchus
Itrsifondonykus
Ikabestei
Utosaurus
Acalosaurus
Uton


Iteration: 970000, Loss: 21.338921

Ilus
Andaadosaurus
Ausprimasaurus
Ieebbosaurus
Utosaurus
A